# Imports


In [1]:
import pandas as pd
import numpy as np 
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
!pip install category_encoders==2.*
import category_encoders as ce 
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin

# for modeling
from sklearn.neighbors import NearestNeighbors
from sklearn.model_selection import train_test_split

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


     |████████████████████████████████| 81kB 2.2MB/s 


# Classes first

In [ ]:
class Songs(DB.Model):
    """ ##############################FOR EXAMPLE ONLY:
    __tablename__ = "Songs"
    id = DB.Column(DB.BigInteger, primary_key=True)
    genre = DB.Column(DB.String(50))
    artist_name = DB.Column(DB.String(50))
    track_name = DB.Column(DB.String(100))
    track_id = DB.Column(DB.String(50))
    popularity = DB.Column(DB.Integer)
    acousticness = DB.Column(DB.Float)
    danceability = DB.Column(DB.Float)
    duration_ms = DB.Column(DB.Integer)
    energy = DB.Column(DB.Float)
    instrumentalness = DB.Column(DB.Float)
    key = DB.Column(DB.Integer)
    liveness = DB.Column(DB.Float)
    loudness = DB.Column(DB.Float)
    mode = DB.Column(DB.Integer)
    speechiness = DB.Column(DB.Float)
    tempo = DB.Column(DB.Float)
    time_signature = DB.Column(DB.Integer)
    valence = DB.Column(DB.Float)

    def __repr__(self):
        return '<Song {}>'.format(self.track_name)
        """
        

In [ ]:
class My_encoder(BaseEstimator, TransformerMixin):
   
    def __init__(self,drop = 'first',sparse=False):
        self.encoder = OneHotEncoder(drop = drop,sparse = sparse)
        self.features_to_encode = []
        self.columns = []
    
    def fit(self,X_train,features_to_encode):
        
        data = X_train.copy()
        self.features_to_encode = features_to_encode
        data_to_encode = data[self.features_to_encode]
        self.columns = pd.get_dummies(data_to_encode,drop_first = True).columns
        self.encoder.fit(data_to_encode)
        return self.encoder
    
    def transform(self,X_test):
        
        data = X_test.copy()
        data.reset_index(drop = True,inplace =True)
        data_to_encode = data[self.features_to_encode]
        data_left = data.drop(self.features_to_encode,axis = 1)
        
        data_encoded = pd.DataFrame(self.encoder.transform(data_to_encode),columns = self.columns)
        
        return pd.concat([data_left,data_encoded],axis = 1)

# Now for the functions: my suggested approach is instantiating the model when we instantiate the app:

In [ ]:
# our functions would sit inside the flask create_app(): function (assuming flask app instantiated)
def create_app(): #notice that all the functions below are part of the app:

  #app = Flask(__name__)

  def pre_process(df):
    #Encoding
    features_to_encode = ['time_signature', 'mode', 'key', 'genre']
    enc= My_encoder()
    enc.fit(df,features_to_encode)
    df_encoded = enc.transform(df)
    #return df_encoded
    #Scaling - after reading up on the spotify method for grading these features, I think we can do without it.
    #scaler = StandardScaler()
    """scaleCols = ['popularity',	'acousticness',	'danceability',	'duration_ms',	'energy',
              'instrumentalness',	'liveness',	'loudness',	'speechiness',	'tempo',	
              'valence',	'time_signature_1/4',	'time_signature_3/4',	'time_signature_4/4',	
              'time_signature_5/4',	'mode_Minor',	'key_A#',	'key_B',	'key_C',
              'key_C#',	'key_D',	'key_D#', 'key_E',	'key_F',	'key_F#',	'key_G',	
              'key_G#',	'genre_Alternative',	'genre_Anime',	'genre_Blues',
              "genre_Children's Music",	"genre_Children’s Music",	'genre_Classical', 
              'genre_Comedy',	'genre_Country',	'genre_Dance',	'genre_Electronic',	'genre_Folk',
              'genre_Hip-Hop',	'genre_Indie',	'genre_Jazz',	'genre_Movie',	'genre_Opera',	
              'genre_Pop',	'genre_R&B',	'genre_Rap',	'genre_Reggae',	'genre_Reggaeton',
              'genre_Rock',	'genre_Ska',	'genre_Soul',	'genre_Soundtrack',	'genre_World']

    df_scaled = scaler.fit_transform(df_encoded[scaleCols])
    """
    return df_encoded


  """ Now Instantiating the Model when we instantiate the app """

  df_encoded = pre_process(df)

  neigh = NearestNeighbors(n_neighbors=15)
  features = ['acousticness', 'danceability', 'energy', 'instrumentalness',
              'liveness', 'loudness', 'speechiness', 'tempo','valence', 
             	'time_signature_1/4',	'time_signature_3/4',	'time_signature_4/4',
            	'time_signature_5/4',	'mode_Minor',	'key_A#',	'key_B',	'key_C',
            	'key_C#',	'key_D',	'key_D#',	'key_E','key_F',	'key_F#',	'key_G',	'key_G#']
  X = df_encoded[features].values
  neigh.fit(X)
  
  '''
        Takes in a dataframe, a feature set array and a song ID and 
        returns a list of tuples of Artist Name and Track Name
    '''
    def closest_ten(df: pd.DataFrame, X_array: np.ndarray, song_id: int) -> List[Tuple]:
        song = df.iloc[song_id]
        X_song = X[song_id]
        _, neighbors = neigh.kneighbors(np.array([X_song]))
        return neighbors[0][1:]
    
    # accepts the cursor and the row as a tuple and returns a dictionary result and you can object column by name 
    def dict_factory(cursor, row):
        d = {}
        for idx, col in enumerate(cursor.description):
            d[col[0]] = row[idx]
        return d

    ''' PROPOSAL:
        Main API route that takes in a Track ID sent from the user and pairs it with
        a unique track_id from the database and returns the closest songs from our
        Machine Learning model.
    '''
    @app.route('/track/<track_id>', methods=['GET'])
    def track(track_id):
        track_id = int(track_id)
        conn = sqlite3.connect('Spotify_Songs.db')
        conn.row_factory = dict_factory
        curs = conn.cursor()
        songlist = []
        song_recs = closest_ten(df, X, track_id)
        for idx in song_recs:
            song = curs.execute(
                f'SELECT DISTINCT * FROM Songs WHERE id=={idx};').fetchone()
            songlist.append(song)
        return jsonify(songlist)

    return app
